In [1]:
# my-program.py
from cicada.communicator import SocketCommunicator

def main(communicator):
    # Your program here
    pass
    
SocketCommunicator.run(world_size=5, fn=main);

In [2]:
def main(communicator):
    return f"Hello from player {communicator.rank}!"

results = SocketCommunicator.run(world_size=5, fn=main)
for rank, result in enumerate(results):
    print(f"Player {rank} result: {result!r}")

Player 0 result: 'Hello from player 0!'
Player 1 result: 'Hello from player 1!'
Player 2 result: 'Hello from player 2!'
Player 3 result: 'Hello from player 3!'
Player 4 result: 'Hello from player 4!'


Special return values are used to indicate players that fail, whether by raising an exception or dying unexpectedly:

In [3]:
import os
import signal

def main(communicator):
    # Examples of normal return values.
    if communicator.rank in [0, 1, 2]:
        return f"Hello from player {communicator.rank}!"
    # Example of a failure that raises an exception.
    if communicator.rank == 3:
        raise RuntimeError("Ahhhh! YOU GOT ME!")
    # Example of a process that dies unexpectedly.
    if communicator.rank == 4:
        os.kill(os.getpid(), signal.SIGKILL)

results = SocketCommunicator.run(world_size=5, fn=main)
for rank, result in enumerate(results):
    print(f"Player {rank} result: {result!r}")

Comm world player 3 failed: RuntimeError('Ahhhh! YOU GOT ME!')
Comm world player 4 failed: Terminated(exitcode=-9)


Player 0 result: 'Hello from player 0!'
Player 1 result: 'Hello from player 1!'
Player 2 result: 'Hello from player 2!'
Player 3 result: Failed(exception=RuntimeError('Ahhhh! YOU GOT ME!'))
Player 4 result: Terminated(exitcode=-9)


In [4]:
from cicada.communicator.socket import Failed

def main(communicator):
    # Examples of normal return values.
    if communicator.rank in [0, 1, 2]:
        return f"Hello from player {communicator.rank}!"
    # Example of a failure that raises an exception.
    if communicator.rank == 3:
        raise RuntimeError("Ahhhh! YOU GOT ME!")
    # Example of a process that dies unexpectedly.
    if communicator.rank == 4:
        os.kill(os.getpid(), signal.SIGKILL)

results = SocketCommunicator.run(world_size=5, fn=main)
for rank, result in enumerate(results):
    print(f"Player {rank} result: {result!r}")
    if isinstance(result, Failed):
        print(f"Player {rank}: {result.traceback}")

Comm world player 3 failed: RuntimeError('Ahhhh! YOU GOT ME!')
Comm world player 4 failed: Terminated(exitcode=-9)


Player 0 result: 'Hello from player 0!'
Player 1 result: 'Hello from player 1!'
Player 2 result: 'Hello from player 2!'
Player 3 result: Failed(exception=RuntimeError('Ahhhh! YOU GOT ME!'))
Player 3: Traceback (most recent call last):
  File "/Users/tshead/src/cicada-mpc/cicada/communicator/socket/__init__.py", line 786, in launch
    result = fn(communicator, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/tl/h2xygzzn1154jzjn_n01x860001l4n/T/ipykernel_7011/308613329.py", line 9, in main
    raise RuntimeError("Ahhhh! YOU GOT ME!")
RuntimeError: Ahhhh! YOU GOT ME!

Player 4 result: Terminated(exitcode=-9)
